In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
def readin_small_data():
    """Read small data set into a 2D numpy array
    return: numpy array contains the small data set, shape: (50,000, 230)
    """
    PATH = os.getcwd()
    DATA_PATH = os.path.join(PATH, 'orange_small_train','orange_small_train.data')
    
    data = pd.read_csv(DATA_PATH, delimiter='\t')
    
    return data


def readin_label(target):
    """Read upselling, churn or appetency label into a 2D numpy array
    parameter target: choose which label to load
    return: numpy array contains the specified label of small data set, shape: (50,000, 1)
    """
    assert target in ['upselling', 'churn', 'appetency']
    PATH = os.getcwd()
    LABEL_PATH = os.path.join(PATH, 'orange_small_train', 'orange_small_train_'+target+'.labels.txt')

    label = pd.read_csv(LABEL_PATH, delimiter='\t')
    label[label==1] = True
    label[label==-1] = False
    
    return label

data = readin_small_data()
print(data.shape)
upselling_label = readin_label('upselling')
print(upselling_label.shape)
churn_label = readin_label('churn')
print(churn_label.shape)
appetency_label = readin_label('appetency')
print(appetency_label.shape)

(50000, 230)
(49999, 1)
(49999, 1)
(49999, 1)


In [8]:
labels = np.hstack([appetency_label, churn_label, upselling_label])
labels = np.hstack([labels, ~np.any(labels, axis=1)[:,np.newaxis]])
labels = labels.astype(int)

In [9]:
missing_rate = data.isnull().sum() / data.shape[0]

numerical_data = data.loc[:, data.columns[:190]]
numerical_missing_rate = numerical_data.isnull().sum() / numerical_data.shape[0]
useful_numerical_data = numerical_data.loc[:, numerical_missing_rate < 0.5]

categorical_data = data.loc[:, data.columns[190:]]
categorical_missing_data = categorical_data.isnull().sum() / categorical_data.shape[0]
useful_categorical_data = categorical_data.loc[:, categorical_missing_data < 0.5]


# sns.distplot(numerical_data.isnull().sum(axis=1) / data.shape[0], kde=False, rug=True, bins=10)
# t=numerical_data.isnull().sum(axis=1) / numerical_data.shape[1]
clean_numerical_data = useful_numerical_data.fillna(useful_numerical_data.mean())
clean_categorical_data = useful_categorical_data.fillna(useful_categorical_data.mode().iloc[0])

In [10]:
clean_numerical_data.as_matrix().shape

(50000, 41)

In [11]:
clean_numerical_data.to_csv('small_clean_numerical_data.csv')
clean_categorical_data.to_csv('small_clean_categorical_data.csv')

In [9]:
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import keras
from sklearn.cross_validation import train_test_split

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=41))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu'))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu'))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

model.add(Dense(units=4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

# trainX, valX, trainY, valY = train_test_split(clean_numerical_data, labels, test_size=0.2, random_state=42)



model.fit(sklearn.preprocessing.robust_scale(clean_numerical_data), labels, epochs=400, batch_size=200, validation_split=0.2)

Using TensorFlow backend.
/afs/inf.ed.ac.uk/user/s17/s1719186/miniconda3/envs/dme/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Train on 40000 samples, validate on 10000 samples
Epoch 1/400
40000/40000 [==============================] - 2s 49us/step - loss: 0.8487 - acc: 0.7754 - val_loss: 0.7005 - val_acc: 0.8324
Epoch 2/400
40000/40000 [==============================] - 1s 15us/step - loss: 0.6633 - acc: 0.8358 - val_loss: 0.6563 - val_acc: 0.8323
Epoch 3/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.6286 - acc: 0.8358 - val_loss: 0.6352 - val_acc: 0.8324
Epoch 4/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.6095 - acc: 0.8358 - val_loss: 0.6208 - val_acc: 0.8324
Epoch 5/400
40000/40000 [==============================] - 1s 16us/step - loss: 0.5962 - acc: 0.8359 - val_loss: 0.6112 - val_acc: 0.8324
Epoch 6/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.5866 - acc: 0.8358 - val_loss: 0.6052 - val_acc: 0.8331
Epoch 7/400
40000/40000 [==============================] - 1s 16us/step - loss: 0.5788 - acc: 0.8365 - val_loss: 0.5981 - 

Epoch 118/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.4407 - acc: 0.8647 - val_loss: 0.5773 - val_acc: 0.8377
Epoch 119/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.4387 - acc: 0.8656 - val_loss: 0.5744 - val_acc: 0.8425
Epoch 120/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.4381 - acc: 0.8659 - val_loss: 0.5762 - val_acc: 0.8406
Epoch 121/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.4377 - acc: 0.8655 - val_loss: 0.5766 - val_acc: 0.8420
Epoch 122/400
40000/40000 [==============================] - 1s 14us/step - loss: 0.4369 - acc: 0.8658 - val_loss: 0.5770 - val_acc: 0.8424
Epoch 123/400
40000/40000 [==============================] - 1s 15us/step - loss: 0.4362 - acc: 0.8656 - val_loss: 0.5779 - val_acc: 0.8379
Epoch 124/400
40000/40000 [==============================] - 1s 15us/step - loss: 0.4362 - acc: 0.8663 - val_loss: 0.5792 - val_acc: 0.8410
Epoch 125/400
40000/

40000/40000 [==============================] - 1s 13us/step - loss: 0.3767 - acc: 0.8799 - val_loss: 0.6432 - val_acc: 0.8357
Epoch 235/400
40000/40000 [==============================] - 0s 12us/step - loss: 0.3757 - acc: 0.8802 - val_loss: 0.6467 - val_acc: 0.8304
Epoch 236/400
40000/40000 [==============================] - 0s 12us/step - loss: 0.3754 - acc: 0.8804 - val_loss: 0.6467 - val_acc: 0.8286
Epoch 237/400
40000/40000 [==============================] - 0s 12us/step - loss: 0.3740 - acc: 0.8814 - val_loss: 0.6474 - val_acc: 0.8352
Epoch 238/400
40000/40000 [==============================] - 0s 12us/step - loss: 0.3740 - acc: 0.8802 - val_loss: 0.6496 - val_acc: 0.8353
Epoch 239/400
40000/40000 [==============================] - 0s 12us/step - loss: 0.3744 - acc: 0.8806 - val_loss: 0.6472 - val_acc: 0.8299
Epoch 240/400
40000/40000 [==============================] - 0s 12us/step - loss: 0.3728 - acc: 0.8807 - val_loss: 0.6476 - val_acc: 0.8320
Epoch 241/400
40000/40000 [=======

40000/40000 [==============================] - 1s 13us/step - loss: 0.3281 - acc: 0.8944 - val_loss: 0.7294 - val_acc: 0.8176
Epoch 351/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.3250 - acc: 0.8961 - val_loss: 0.7300 - val_acc: 0.8179
Epoch 352/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.3258 - acc: 0.8953 - val_loss: 0.7350 - val_acc: 0.8148
Epoch 353/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.3254 - acc: 0.8960 - val_loss: 0.7340 - val_acc: 0.8217
Epoch 354/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.3261 - acc: 0.8958 - val_loss: 0.7364 - val_acc: 0.8224
Epoch 355/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.3260 - acc: 0.8953 - val_loss: 0.7331 - val_acc: 0.8188
Epoch 356/400
40000/40000 [==============================] - 1s 13us/step - loss: 0.3237 - acc: 0.8959 - val_loss: 0.7377 - val_acc: 0.8257
Epoch 357/400
40000/40000 [=======